<a href="https://colab.research.google.com/github/rtrochepy/astronomer/blob/main/supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Cargar el conjunto de datos
df = pd.read_csv("processed_dataset_to_train_models.csv")

# Mostrar las columnas del DataFrame
print("Columnas del DataFrame:", df.columns)

# Definir la columna objetivo
target_column = 'label'

# Definir las columnas de características (SelectKBest y RFE)
selectKBest_columns = ['Payment_6804', 'Base_80863', 'Infraction_QJJF', 'Base_76065',
       'Infraction_TLPJ', 'Base_1165', 'Base_39598', 'Base_85131', 'Base_9516',
       'Infraction_BSU', 'Infraction_ZYW', 'Infraction_TBP', 'Infraction_PBC',
       'Base_0229', 'Base_69608', 'Base_3041', 'Infraction_QKZN',
       'Infraction_CZE', 'Base_9103', 'Base_67254_low', 'label']
rfe_columns = ['Payment_6804', 'Base_80863', 'Expenditure_JIG', 'Base_02683',
       'Infraction_ZWWJ', 'Infraction_QJJF', 'Infraction_EJZ',
       'Infraction_GGO', 'Infraction_TLPJ', 'Base_1165', 'Base_39598',
       'Base_6187', 'Base_85131', 'Risk_9995', 'Infraction_AYWV', 'Base_9516',
       'Expenditure_HMO', 'Infraction_BSU', 'Infraction_ZYW', 'Infraction_TBP',
       'Infraction_PBC', 'Base_0229', 'Base_69608', 'Base_3041',
       'Infraction_QKZN', 'Infraction_CZE', 'Expenditure_MTRQ',
       'Infraction_RKTA', 'Infraction_KEJT', 'label']


# Función para entrenar, escalar y evaluar modelos
def train_and_evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    # Escalado de los datos
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entrenamiento y evaluación
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return model_name, acc

# Función principal para evaluar diferentes modelos
def evaluate_models_on_datasets(datasets, target_column, test_size=0.3):
    # Modelos de clasificación
    models = [
        (LogisticRegression(random_state=42, max_iter=1000), "Regresión Logística"),
        (DecisionTreeClassifier(random_state=42), "Árbol de Decisión"),
        (KNeighborsClassifier(), "k-Nearest Neighbors (k-NN)"),
        (SVC(kernel='linear', random_state=42), "Máquina de Soporte Vectorial (SVM)")
    ]

    for df, method_name in datasets:
        print(f"\n=== Evaluación utilizando {method_name} ===")

        # Separar características (X) y variable objetivo (y)
        X = df.drop(columns=[target_column])
        y = df[target_column]

        # Dividir los datos en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42
        )

        # Evaluar cada modelo y almacenar resultados
        results = [
            train_and_evaluate_model(model, X_train, X_test, y_train, y_test, model_name)
            for model, model_name in models
        ]

        # Ordenar los resultados por accuracy de mayor a menor
        results_sorted = sorted(results, key=lambda x: x[1], reverse=True)

        # Imprimir resultados ordenados
        print("Resultados:")
        for name, acc in results_sorted:
            print(f"{name}: Accuracy = {acc:.4f}")

# Crear listas de DataFrames y nombres para iterar
target_column = 'label'
datasets = [(df_SelectKBest, 'SelectKBest'), (df_rfe, 'RFE')]

# Ejecutar la evaluación
evaluate_models_on_datasets(datasets, target_column)

Columnas del DataFrame: Index(['Payment_6804', 'Base_4569', 'Infraction_QKZN', 'Infraction_GGO',
       'Base_76065', 'Infraction_BSU', 'Expenditure_MTRQ', 'Expenditure_HMO',
       'Base_9516', 'Infraction_TLPJ', 'Base_1165', 'Base_85131', 'Base_3041',
       'Infraction_PBC', 'Infraction_RKTA', 'Expenditure_JIG',
       'Infraction_ZMKI', 'Base_0229', 'Infraction_ZTNC', 'Base_39598',
       'Infraction_CZE', 'Base_67254_low', 'Base_02683', 'Infraction_TBP',
       'Infraction_AYWV', 'Infraction_ZWWJ', 'Base_80863', 'Infraction_EJZ',
       'Infraction_QJJF', 'Base_6187', 'Base_2810', 'Base_69608', 'Risk_9995',
       'Infraction_KEJT', 'Infraction_ZYW', 'Base_9103', 'label'],
      dtype='object')

=== Evaluación utilizando SelectKBest ===
Resultados:
k-Nearest Neighbors (k-NN): Accuracy = 0.9082
Árbol de Decisión: Accuracy = 0.8766
Máquina de Soporte Vectorial (SVM): Accuracy = 0.8343
Regresión Logística: Accuracy = 0.8332

=== Evaluación utilizando RFE ===
Resultados:
k-Nearest Nei